In [ ]:
import pandas as pd
import glob
import json
import dotted # https://pypi.org/project/dotted-notation/
import re
import matplotlib.pyplot as plt
import itertools
import numpy as np

from pathlib import Path
import seaborn as sns
import lib.datasciencetoolbelt as dstools
from lib.resultstorage import ResultStorage

In [ ]:
dstools.setup({
    "seaborn_context": "talk",
    "savefig": {
        "enable": True,
        "dir": Path("./postprocess_results"),
    }
})
result_storage = ResultStorage(Path("./results"))

#%matplotlib qt
%matplotlib inline


In [ ]:
result_storage_prefix = "blockdev_comparison_v1"


def get_fio_write_metrics(output_json):
    d = output_json
    jobs = dotted.get(d, "fio_jsonplus.jobs")
    assert len(jobs) == 1
    j0 = jobs[0]
    jw = jobs[0]["write"]
    return jw


def to_fio_results_dict(output_json):
    jw = get_fio_write_metrics(output_json)
    return {
        "w_iops_mean": jw["iops_mean"],
        "w_iops_stddev": jw["iops_stddev"],
        "w_lat_mean": dotted.get(jw, "lat_ns.mean"),
        "w_lat_stddev": dotted.get(jw, "lat_ns.stddev"),
    }

def to_row(d):
    
    blockdev_stack = dotted.get(d, 'storage_stack.identity')
    
    fio = to_fio_results_dict(dotted.get(d, 'fio_result'))
    
    return {
        "blockdev_stack": blockdev_stack,
        "numjobs": dotted.get(d, 'fio_result.fio_config.numjobs'),
        **fio,
    }

rows = [to_row(j) for j in result_storage.iter_results(result_storage_prefix)]
rows = list(filter(lambda d: d is not None, rows))
rows = list(itertools.chain(rows))
df = pd.DataFrame(rows)
df


In [ ]:
data = df.copy()
data['unused'] = 1

data = data[data.blockdev_stack.map(lambda v: "nc_2" not in v)]
# data = data[data.blockdev_stack.map(lambda v: "byp_1" not in v)]
# data = data[data.blockdev_stack.map(lambda v: v in ['native', 'devpmem', 'dm-writecache'])]

data = data.set_index(['blockdev_stack', 'numjobs', 'unused'])

In [ ]:
data['w_iops_mean'].unstack('blockdev_stack').sort_index().plot(legend=True, figsize=(15, 15))